# Bert Transfer Learning 
### Incorporated using Hugginface 
### Code was executed and ran in Google Colab 

In [1]:
!pip install transformers==4.6.0
!pip install datasets

     |████████████████████████████████| 2.3 MB 12.6 MB/s 
     |████████████████████████████████| 3.3 MB 33.2 MB/s 
     |████████████████████████████████| 895 kB 43.8 MB/s 
     |████████████████████████████████| 270 kB 11.7 MB/s 
     |████████████████████████████████| 125 kB 48.8 MB/s 
     |████████████████████████████████| 1.3 MB 49.6 MB/s 
     |████████████████████████████████| 243 kB 45.6 MB/s 
     |████████████████████████████████| 56 kB 4.2 MB/s 
     |████████████████████████████████| 160 kB 44.9 MB/s 
     |████████████████████████████████| 271 kB 49.3 MB/s 
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.8
    Uninstalling huggingface-hub-0.0.8:
      Successfully uninstalled huggingface-hub-0.0.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.6.0 requires huggingface-hub==0.0.8, but you

In [2]:
from tensorflow import keras
import tensorflow as tf
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import BertTokenizerFast, BertForSequenceClassification,TFAutoModelForSequenceClassification,TFGPT2ForSequenceClassification,GPT2TokenizerFast,GPT2Config,DistilBertTokenizer,TFTrainer,TFTrainingArguments
from transformers import Trainer, TrainingArguments
import numpy as np
import random
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from sklearn.metrics import accuracy_score
max_length = 150


Bert was chosen as pretrained model and will now be fine-tuned

In [3]:

from transformers import TFDistilBertForSequenceClassification, DistilBertConfig
import tensorflow as tf

distil_bert = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizer.from_pretrained(distil_bert, do_lower_case=True, add_special_tokens=True,max_length=max_length, pad_to_max_length=True)
tokenizer.pad_token = "[PAD]"
config = DistilBertConfig(num_labels=2)
config.output_hidden_states = False


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [4]:
def create_splits(data,test_size=0.2):
  text = data['text']
  labels = data['out']
  return train_test_split(text,labels,test_size = test_size)

data = load_dataset('csv', data_files='/content/modeling_data.csv')
(train_texts,test_texts, train_labels,test_labels)= create_splits(data['train'])
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings),train_labels))

Using custom data configuration default-564265955e6db201


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-564265955e6db201/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model.summary()

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_layer_norm', 'vocab_projector', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
distilbert (TFDistilBertMain multiple                  66362880  
_________________________________________________________________
pre_classifier (Dense)       multiple                  590592    
_________________________________________________________________
classifier (Dense)           multiple                  1538      
_________________________________________________________________
dropout_19 (Dropout)         multiple                  0         
Total params: 66,955,010
Trainable params: 66,955,010
Non-trainable params: 0
_________________________________________________________________


In [6]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),metrics = 'accuracy') 

In [7]:
model.fit(train_dataset.shuffle(1000).batch(16), epochs=3, batch_size=16)

Epoch 1/3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autogr

In [8]:
# functio to make predictions over list of text 
# Takes care of preprocessing, all the way to prediction 
def pred_vectorized(text,model = model):
    preds = tokenizer.encode(text,truncation=True,padding=True,max_length=max_length,return_tensors="tf")
    pred =  model.predict(preds)[0]
    softed = tf.nn.softmax(pred).numpy()[0]
    out = np.where( np.round(softed) ==1)[0][0]
    return out

In [9]:
y_hat = [pred_vectorized(x) for x in test_texts]

In [10]:
accuracy_score(test_labels,y_hat)

0.948509485094851

Prediction method works and we also have a great accuracy score. 
For the base incorportation of this app we will continue with this model.

In [11]:
model.save("/content/model")

INFO:tensorflow:Assets written to: /content/model/assets


INFO:tensorflow:Assets written to: /content/model/assets
/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [12]:
!zip -r /content/model.zip /content/model
from google.colab import files
files.download("/content/model.zip")

  adding: content/model/ (stored 0%)
  adding: content/model/saved_model.pb (deflated 92%)
  adding: content/model/variables/ (stored 0%)
  adding: content/model/variables/variables.index (deflated 79%)
  adding: content/model/variables/variables.data-00000-of-00001 (deflated 27%)
  adding: content/model/keras_metadata.pb (deflated 94%)
  adding: content/model/assets/ (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>